In [16]:
import os
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from scipy import stats

from visdatcompy.common.utils import scan_directory

In [20]:
def ImageLoad(image, echo='off'):
    if echo == 'on':
        print('Loading image ' + image + '...')

    img = cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2RGB)
    
    # Получение текущих размеров изображения
    height, width = img.shape[:2]
    
    # Вычисление новых размеров, сохраняя пропорции
    new_width = 512
    new_height = int(height * (new_width / width))
    
    # Масштабирование изображения до новых размеров
    resized_img = cv2.resize(img, (new_width, new_height))

    return resized_img

descs_time_start = time.time()

feature_extractor = cv2.SIFT_create()

t = 0
Xt = np.zeros((10000000,128))
yt = np.zeros((10000000,))

images_path = scan_directory(r"C:\Users\sharj\Desktop\STUDY\visdatcompy_datasets\small_drone_test_compressed")

images_fpath = list(map(lambda x: os.path.join(x[0], x[1]), images_path))

N = len(images_fpath)
print (images_fpath)

for i in range(N):
  I = ImageLoad(images_fpath[i], echo='on')
  J = cv2.cvtColor(I, cv2.COLOR_RGB2GRAY)
  # SIFT extraction
  kp, desc = feature_extractor.detectAndCompute(J, None)
  ni = desc.shape[0]
  for j in range(ni):
    f = desc[j,:]
    Xt[t,:] =  f/np.linalg.norm(f)
    yt[t] = i
    t = t+1

  print('... '+str(ni)+' Дескрипторы были извлечены.')

X = Xt[0:t,:]
y = yt[0:t]
print('Количество дескрипторов SIFT в '+str(N)+' images: '+str(t))

descs_time_end = time.time()
descs_time = descs_time_end - descs_time_start

print(f'Общее время выполнения: {descs_time}')

['C:\\Users\\sharj\\Desktop\\STUDY\\visdatcompy_datasets\\small_drone_test_compressed\\0_1.jpg', 'C:\\Users\\sharj\\Desktop\\STUDY\\visdatcompy_datasets\\small_drone_test_compressed\\0_2.jpg', 'C:\\Users\\sharj\\Desktop\\STUDY\\visdatcompy_datasets\\small_drone_test_compressed\\0_3.jpg', 'C:\\Users\\sharj\\Desktop\\STUDY\\visdatcompy_datasets\\small_drone_test_compressed\\0_4.jpg', 'C:\\Users\\sharj\\Desktop\\STUDY\\visdatcompy_datasets\\small_drone_test_compressed\\1_1.jpg', 'C:\\Users\\sharj\\Desktop\\STUDY\\visdatcompy_datasets\\small_drone_test_compressed\\1_2.jpg', 'C:\\Users\\sharj\\Desktop\\STUDY\\visdatcompy_datasets\\small_drone_test_compressed\\1_3.jpg', 'C:\\Users\\sharj\\Desktop\\STUDY\\visdatcompy_datasets\\small_drone_test_compressed\\2_1.jpg', 'C:\\Users\\sharj\\Desktop\\STUDY\\visdatcompy_datasets\\small_drone_test_compressed\\2_2.jpg', 'C:\\Users\\sharj\\Desktop\\STUDY\\visdatcompy_datasets\\small_drone_test_compressed\\2_3.jpg', 'C:\\Users\\sharj\\Desktop\\STUDY\\visd

In [22]:
def find_similar_image(itest, X, y):
    ik = itest
    ii = np.where(y==ik)[0]  
    jj = np.where(y!=ik)[0]  

    Xi = X[ii,:]  
    Xj = X[jj,:]  
    yj = y[jj]    

    Dt = np.dot(Xj,Xi.T)

    n = Xi.shape[0]

    z = np.zeros((n,))

    # d = np.zeros((n,))
    d = []

    for k in range(n):
        h = Dt[:,k]         
        i = h.max()         
        z[k] = i
        j = np.where(h==i)[0]  

        # d[k] = yj[j]  
        d.append(yj[j[0]])

    kk = np.where(z > 0.9)

    # m = stats.mode(d[kk])
    m = stats.mode(np.array(d)[kk])
    
    ifound = int(m[0]) 

    return ifound


def visualize_similar_images(itest):
    ifound = find_similar_image(itest, X, y)
    I = ImageLoad(images_fpath[itest],echo='on')
    print('Тестовое изображение: '+str(itest))

    plt.imshow(I,cmap='gray')
    plt.axis('off')
    plt.show()
    print(' ')

    J = ImageLoad(images_fpath[ifound],echo='on')
    print('Для изображения '+str(itest)+', наиболее похожее изображение: '+str(ifound)+'.')
    plt.imshow(J,cmap='gray')
    plt.axis('off')
    plt.show()
    print('--------------------------------------------------')

# Пример использования с ipywidgets interact
interact(visualize_similar_images, itest=(0, 38))


interactive(children=(IntSlider(value=19, description='itest', max=38), Output()), _dom_classes=('widget-inter…

<function __main__.visualize_similar_images(itest)>